## 查看defectsj patch

In [ ]:
import json
file='input_base.json'
with open(file, 'r') as file:
    data1 = json.load(file)['data']
    for i in data1:
        print(f'----------{i}----------')
        print(i)
        print(data1[i]['buggy_lines'])
        print(data1[i]['patch'])

## codellama infill dataset

In [19]:
import json
file_name='codellama_7b_infill.json'

with open('input_base.json', 'r') as file:
    data = json.load(file)

In [20]:
for key in data['data']:
    # format as "<PRE> {pre} <SUF>{suf} <MID>"
    data['data'][key]['input']="<PRE> "+data['data'][key]['pre']+" <SUF>"+data['data'][key]['post']+ " <MID>"

json.dump(data, open(file_name, 'w'), indent=2)

In [ ]:
with open(file_name, 'r') as file:
    data1 = json.load(file)['data']
    for i in data1:
        print(f'----------{i}----------')
        print(i)
        print(data1[i]['input'])